In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from pathlib import Path

# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('headless')

try:
    # driver = webdriver.Chrome(options=chrome_options)
    driver = webdriver.Chrome()
except:
    service = Service(ChromeDriverManager().install())
    # driver = webdriver.Chrome(service=service, options=chrome_options)
    driver = webdriver.Chrome(service=service)

df = pd.read_excel('buscas.xlsx')
display(df)

In [ ]:
def google_search(driver, product_name, forbidden_words, min_price, max_price):

    driver.get('https://www.google.com/')
    driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(f'{df["Nome"][0]}')
    driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)


    elements_list = driver.find_elements(By.CLASS_NAME, 'hdtb-mitem')
    for element in elements_list:
            if 'shopping' in element.text.lower():
                element.click()
                break


    result_list = driver.find_elements(By.CLASS_NAME, 'sh-dgr__grid-result')

    # Dealing with input variables
    product_name = product_name.lower()
    product_name_list = product_name.split(" ")

    forbidden_words = forbidden_words.lower()
    forbidden_words_list = forbidden_words.split(" ")

    max_price = float(max_price)
    min_price = float(min_price)

    found_products = [] # the final result
    for result in result_list:
        name = result.find_element(By.CLASS_NAME, 'tAxDx').text
        name = name.lower()

        # Verifying the product name
        has_forbidden_words = False
        for word in forbidden_words_list:
            if word in name:
                has_forbidden_words = True
        
        # Ensure that the product name is the desired one
        has_all_product_names = True
        for word in product_name_list:
            if word not in name:
                has_all_product_names = False
        
        if not has_forbidden_words and has_all_product_names: # verificando o nome
            try:
                # Formating the price -> (from str to float)
                price = float(result.find_element(By.CLASS_NAME, 'a8Pemb').text.replace("R$ ", "").replace(".", "").replace(",", "."))
        
                # Checking if price is within the defined limit
                if min_price <= price <= max_price:
                    link = result.find_element(By.CSS_SELECTOR, 'a.shntl').get_attribute('href')
                    found_products.append((name, price, link))
            except:
                continue


    return found_products



In [ ]:
product_name = 'iphone 12 64gb'
forbidden_words = 'mini watch 11'
min_price = 3000
max_price = 3500

final_results = google_search(driver, product_name, forbidden_words, min_price, max_price)

for i in final_results:
    print(i)